In [1]:
import pandas as pd
import re
import os
os.getcwd()

'/home/user02/박광렬조 프로젝트'

In [2]:
os.listdir(path='./L_POINT data')

['total_weather.csv',
 'session_after.csv',
 'product_after.csv',
 'L_point_weather.csv',
 'week.csv',
 'dust.csv',
 'Search1.csv',
 'search2_after.csv',
 '.ipynb_checkpoints',
 'Custom.csv',
 'Master.csv']

In [3]:
dust = pd.read_csv("./L_POINT data/dust.csv")
custom = pd.read_csv("./L_POINT data/Custom.csv")
master = pd.read_csv("./L_POINT data/Master.csv")
product = pd.read_csv("./L_POINT data/product_after.csv")
search1 = pd.read_csv("./L_POINT data/Search1.csv")
search2 = pd.read_csv("./L_POINT data/search2_after.csv")
session = pd.read_csv("./L_POINT data/session_after.csv")
weather = pd.read_csv("./L_POINT data/L_point_weather.csv")

In [4]:
dust.head()

,ZON_NM,DATE,px_mean,px_max
0,Gangwon-do,20180301,26.695652,47
1,Gangwon-do,20180302,49.000000,63
2,Gangwon-do,20180303,44.000000,60
3,Gangwon-do,20180304,18.739130,43
4,Gangwon-do,20180305,9.863636,33


In [5]:
weather.head()

,ZON_NM,DATE,WEATHER,T_MAX,T_MIN
0,Gangwon-do,20180301,비,-2.0,4.0
1,Gangwon-do,20180302,구름조금,-6.0,5.0
2,Gangwon-do,20180303,구름조금,-1.0,17.0
3,Gangwon-do,20180304,비,7.0,16.0
4,Gangwon-do,20180305,비/진눈개비,0.0,9.0


In [6]:
weather_total = weather.merge(dust, on= ['ZON_NM','DATE'], how='left')

# 미세먼지 농도 (max) 지수 범주화 
## 출처 http://aqicn.org/here/

In [7]:
px_result = []
for px in weather_total.px_max:
    if type(px) == float:
        if float(px) <= 50 :
            px_result.append("좋음")
        elif  float(px) <= 100 :
            px_result.append("보통")
        elif  float(px) <= 150 :
            px_result.append("민감")
        elif  float(px) <= 200 :
            px_result.append("나쁨")
        elif  float(px) <= 300 :
            px_result.append("매우나쁨")
        elif float(px) <= 3000:
            px_result.append("위험")
        else :
            px_result.append("NA")

In [8]:
weather_total['px_px']=px_result

In [9]:
weather_total.head()

,ZON_NM,DATE,WEATHER,T_MAX,T_MIN,px_mean,px_max,px_px
0,Gangwon-do,20180301,비,-2.0,4.0,26.695652,47.0,좋음
1,Gangwon-do,20180302,구름조금,-6.0,5.0,49.000000,63.0,보통
2,Gangwon-do,20180303,구름조금,-1.0,17.0,44.000000,60.0,보통
3,Gangwon-do,20180304,비,7.0,16.0,18.739130,43.0,좋음
4,Gangwon-do,20180305,비/진눈개비,0.0,9.0,9.863636,33.0,좋음


# 특정 문자만 잡아내기 

In [10]:
import re
pattern = '황사'
r = re.compile(pattern)
result=[]
for text in weather_total.WEATHER:
    match = r.search(text)

    if match:
        startIndex = match.start()
        endIndex = match.end()
        result.append(text[startIndex:endIndex])
    else:
        result.append("NA")
pattern = '비'
r = re.compile(pattern)
result2=[]
for text in weather_total.WEATHER:
    match = r.search(text)

    if match:
        startIndex = match.start()
        endIndex = match.end()
        result2.append(text[startIndex:endIndex])
    else:
        result2.append("NA")

In [11]:
weather_total['황사']=result

In [12]:
weather_total['비']=result2

In [13]:
weather_total[weather_total.비!='NA'].head()

,ZON_NM,DATE,WEATHER,T_MAX,T_MIN,px_mean,px_max,px_px,황사,비
0,Gangwon-do,20180301,비,-2.0,4.0,26.695652,47.0,좋음,NA,비
3,Gangwon-do,20180304,비,7.0,16.0,18.739130,43.0,좋음,NA,비
4,Gangwon-do,20180305,비/진눈개비,0.0,9.0,9.863636,33.0,좋음,NA,비
6,Gangwon-do,20180307,비,1.0,10.0,24.375000,49.0,좋음,NA,비
7,Gangwon-do,20180308,눈/비,1.0,8.0,11.375000,24.0,좋음,NA,비


In [14]:
weather_total[weather_total.황사!='NA']

,ZON_NM,DATE,WEATHER,T_MAX,T_MIN,px_mean,px_max,px_px,황사,비
281,Gyeonggi-do,20180406,흐림/비/안개/황사,4.0,9.0,82.045455,269.0,매우나쁨,황사,비
329,Gyeonggi-do,20180524,맑음/안개/황사,10.0,25.0,90.833333,141.0,민감,황사,NA
780,Gyeongsangbuk-do,20180415,구름조금/황사,9.0,19.0,123.272727,289.0,매우나쁨,황사,NA
781,Gyeongsangbuk-do,20180416,구름조금/황사,9.0,15.0,48.375000,86.0,보통,황사,NA
1016,Gwangju,20180406,구름많음/비/안개/황사,5.0,13.0,64.869565,155.0,나쁨,황사,비
1025,Gwangju,20180415,구름조금/안개/황사,7.0,16.0,109.041667,288.0,매우나쁨,황사,NA
1261,Daegu,20180406,구름많음/비/황사,6.0,16.0,54.666667,157.0,나쁨,황사,비
1270,Daegu,20180415,구름조금/황사,9.0,18.0,127.590909,292.0,매우나쁨,황사,NA
1506,Daejeon,20180406,흐림/비/안개/진눈개비/황사,4.0,13.0,85.916667,266.0,매우나쁨,황사,비
1515,Daejeon,20180415,구름조금/안개/황사,6.0,17.0,87.500000,211.0,매우나쁨,황사,NA


In [15]:
#weather_total.to_csv("total_weather.csv", index=False)

# 다른 데이터 merge 작업

In [16]:
#product.to_csv("product_after.csv", index=False)

In [17]:
session.head()

,CLNT_ID,SESS_ID,SESS_SEQ,SESS_DT,TOT_PAG_VIEW_CT,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM
0,5873599,8641867,9,20180509,82.0,1890.0,mobile,Daejeon,Daejeon
1,5873599,6616320,21,20180611,105.0,1604.0,mobile,Busan,Busan
2,5873599,5886172,40,20180624,41.0,632.0,mobile,Daejeon,Daejeon
3,5873884,1050889,15,20180913,160.0,1035.0,mobile,Gyeonggi-do,Anyang
4,5874461,10298270,5,20180412,13.0,298.0,mobile,Seoul,Seoul


In [18]:
sess_wth = session.merge(weather_total, left_on=['SESS_DT','ZON_NM'], right_on=['DATE','ZON_NM'])

In [19]:
sess_wth.drop(['CITY_NM','DVC_CTG_NM','TOT_PAG_VIEW_CT','TOT_SESS_HR_V','px_mean','SESS_SEQ'], axis=1, inplace=True)

In [20]:
sess_wth.head()

,CLNT_ID,SESS_ID,SESS_DT,ZON_NM,DATE,WEATHER,T_MAX,T_MIN,px_max,px_px,황사,비
0,5873599,8641867,20180509,Daejeon,20180509,맑음,9.0,23.0,50.0,좋음,NA,NA
1,6139048,8646190,20180509,Daejeon,20180509,맑음,9.0,23.0,50.0,좋음,NA,NA
2,6472180,8628644,20180509,Daejeon,20180509,맑음,9.0,23.0,50.0,좋음,NA,NA
3,4747936,8652426,20180509,Daejeon,20180509,맑음,9.0,23.0,50.0,좋음,NA,NA
4,4759574,8622778,20180509,Daejeon,20180509,맑음,9.0,23.0,50.0,좋음,NA,NA


In [21]:
product.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT
0,4139680,7605037,12,642112,색상:워터멜론,[바비브라운],39000.0,1.0
1,4140076,10189797,13,570603,색상:BLK0_(BLK0)BLACK|사이즈:120 / 2개,데상트,39000.0,2.0
2,4142395,6158159,85,179538,(not set),[아베다],39000.0,1.0
3,4144914,7935714,12,554336,색상:블랙|사이즈:160cm(12~13세) / 1개,아디다스 키즈,39000.0,1.0
4,4144917,6406509,78,190306,5개,데코르테,39000.0,5.0


In [22]:
project_main_data = product.merge(sess_wth,on=['CLNT_ID','SESS_ID'])

In [23]:
project_main_data.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_DT,ZON_NM,DATE,WEATHER,T_MAX,T_MIN,px_max,px_px,황사,비
0,4139680,7605037,12,642112,색상:워터멜론,[바비브라운],39000.0,1.0,20180526,Seoul,20180526,맑음/안개,14.0,30.0,126.0,민감,NA,NA
1,4139680,7605037,12,401577,색상:몰리 와우,[바비브라운],39000.0,1.0,20180526,Seoul,20180526,맑음/안개,14.0,30.0,126.0,민감,NA,NA
2,4139680,7605037,28,401577,색상:몰리 와우,[바비브라운],39000.0,1.0,20180526,Seoul,20180526,맑음/안개,14.0,30.0,126.0,민감,NA,NA
3,4139680,7605037,28,642112,색상:카바나,[바비브라운],39000.0,1.0,20180526,Seoul,20180526,맑음/안개,14.0,30.0,126.0,민감,NA,NA
4,4139680,7605037,12,642112,색상:카바나,[바비브라운],39000.0,1.0,20180526,Seoul,20180526,맑음/안개,14.0,30.0,126.0,민감,NA,NA


In [24]:
project_main_data.drop(['HITS_SEQ','PD_ADD_NM','PD_BRA_NM'], axis=1, inplace=True)

In [25]:
project_main_data.head()

,CLNT_ID,SESS_ID,PD_C,PD_BUY_AM,PD_BUY_CT,SESS_DT,ZON_NM,DATE,WEATHER,T_MAX,T_MIN,px_max,px_px,황사,비
0,4139680,7605037,642112,39000.0,1.0,20180526,Seoul,20180526,맑음/안개,14.0,30.0,126.0,민감,NA,NA
1,4139680,7605037,401577,39000.0,1.0,20180526,Seoul,20180526,맑음/안개,14.0,30.0,126.0,민감,NA,NA
2,4139680,7605037,401577,39000.0,1.0,20180526,Seoul,20180526,맑음/안개,14.0,30.0,126.0,민감,NA,NA
3,4139680,7605037,642112,39000.0,1.0,20180526,Seoul,20180526,맑음/안개,14.0,30.0,126.0,민감,NA,NA
4,4139680,7605037,642112,39000.0,1.0,20180526,Seoul,20180526,맑음/안개,14.0,30.0,126.0,민감,NA,NA


In [26]:
project_main = project_main_data.merge(master,on='PD_C')

In [27]:
project_main.head()

,CLNT_ID,SESS_ID,PD_C,PD_BUY_AM,PD_BUY_CT,SESS_DT,ZON_NM,DATE,WEATHER,T_MAX,T_MIN,px_max,px_px,황사,비,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM
0,4139680,7605037,642112,39000.0,1.0,20180526,Seoul,20180526,맑음/안개,14.0,30.0,126.0,민감,NA,NA,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
1,4139680,7605037,642112,39000.0,1.0,20180526,Seoul,20180526,맑음/안개,14.0,30.0,126.0,민감,NA,NA,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
2,4139680,7605037,642112,39000.0,1.0,20180526,Seoul,20180526,맑음/안개,14.0,30.0,126.0,민감,NA,NA,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
3,4139680,7605037,642112,39000.0,1.0,20180526,Seoul,20180526,맑음/안개,14.0,30.0,126.0,민감,NA,NA,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
4,4350334,2023661,642112,39000.0,1.0,20180827,Seoul,20180827,흐림/비/이슬비,20.0,25.0,37.0,좋음,NA,비,크러쉬드 립칼라 - 플럼,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터


In [28]:
p = re.compile("^[가-힇]*")
result=[]
for i in project_main.WEATHER:
    m = p.match(i)
    result.append(m.group())
project_main["WEATHER"]=result

In [29]:
project_main.drop(['PD_NM'], axis=1, inplace=True)

In [30]:
project_main.groupby(['WEATHER']).count()

,CLNT_ID,SESS_ID,PD_C,PD_BUY_AM,PD_BUY_CT,SESS_DT,ZON_NM,DATE,T_MAX,T_MIN,px_max,px_px,황사,비,CLAC1_NM,CLAC2_NM,CLAC3_NM
WEATHER,,,,,,,,,,,,,,,,,
구름많음,1342023,1342023,1342023,1342023,1342023,1342023,1342023,1342023,1342023,1342023,1331118,1342023,1342023,1342023,1342023,1342023,1342023
구름조금,1364662,1364662,1364662,1364662,1364662,1364662,1364662,1364662,1364662,1364662,1364662,1364662,1364662,1364662,1364662,1364662,1364662
눈,812,812,812,812,812,812,812,812,812,812,812,812,812,812,812,812,812
맑음,860123,860123,860123,860123,860123,860123,860123,860123,860123,860123,860123,860123,860123,860123,860123,860123,860123
비,118055,118055,118055,118055,118055,118055,118055,118055,118055,118055,117204,118055,118055,118055,118055,118055,118055
흐림,1339231,1339231,1339231,1339231,1339231,1339231,1339231,1339231,1339231,1339231,1338253,1339231,1339231,1339231,1339231,1339231,1339231


In [31]:
project_main.head()

,CLNT_ID,SESS_ID,PD_C,PD_BUY_AM,PD_BUY_CT,SESS_DT,ZON_NM,DATE,WEATHER,T_MAX,T_MIN,px_max,px_px,황사,비,CLAC1_NM,CLAC2_NM,CLAC3_NM
0,4139680,7605037,642112,39000.0,1.0,20180526,Seoul,20180526,맑음,14.0,30.0,126.0,민감,NA,NA,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
1,4139680,7605037,642112,39000.0,1.0,20180526,Seoul,20180526,맑음,14.0,30.0,126.0,민감,NA,NA,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
2,4139680,7605037,642112,39000.0,1.0,20180526,Seoul,20180526,맑음,14.0,30.0,126.0,민감,NA,NA,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
3,4139680,7605037,642112,39000.0,1.0,20180526,Seoul,20180526,맑음,14.0,30.0,126.0,민감,NA,NA,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터
4,4350334,2023661,642112,39000.0,1.0,20180827,Seoul,20180827,흐림,20.0,25.0,37.0,좋음,NA,비,화장품/뷰티케어,메이크업,블러셔/쉐이딩/하이라이터


# 비가 조금이라도 내렸을 경우를 강우 상황이라고 했을때 무려 40%이므로 이상한 결과가 나옴, 맑음, 구름조금 일경우 강우 상황을 없음으로 정의내려봄

In [32]:
(1>2 and 1<2)

False

In [33]:
result=[]
for i,j in zip(project_main.WEATHER, project_main.비):
    if (i == "맑음" or i =="구름조금") and j =='비':
        result.append("NA")
    elif j=='비':
        result.append("비")
    else :
        result.append("NA")
        
project_main.비 = result

In [34]:
project_main.groupby(['비']).count()

,CLNT_ID,SESS_ID,PD_C,PD_BUY_AM,PD_BUY_CT,SESS_DT,ZON_NM,DATE,WEATHER,T_MAX,T_MIN,px_max,px_px,황사,CLAC1_NM,CLAC2_NM,CLAC3_NM
비,,,,,,,,,,,,,,,,,
NA,3190528,3190528,3190528,3190528,3190528,3190528,3190528,3190528,3190528,3190528,3190528,3183992,3190528,3190528,3190528,3190528,3190528
비,1834378,1834378,1834378,1834378,1834378,1834378,1834378,1834378,1834378,1834378,1834378,1828180,1834378,1834378,1834378,1834378,1834378


In [35]:
project_main[project_main.비 =='비'].describe(include = "all")

,CLNT_ID,SESS_ID,PD_C,PD_BUY_AM,PD_BUY_CT,SESS_DT,ZON_NM,DATE,WEATHER,T_MAX,T_MIN,px_max,px_px,황사,비,CLAC1_NM,CLAC2_NM,CLAC3_NM
count,1.834378e+06,1.834378e+06,1.834378e+06,1.834378e+06,1.834378e+06,1.834378e+06,1834378,1.834378e+06,1834378,1.834378e+06,1.834378e+06,1.828180e+06,1834378,1834378,1834378,1834378,1834378,1834378
unique,NaN,NaN,NaN,NaN,NaN,NaN,16,NaN,4,NaN,NaN,NaN,6,2,1,37,128,893
top,NaN,NaN,NaN,NaN,NaN,NaN,Seoul,NaN,흐림,NaN,NaN,NaN,좋음,NA,비,여성의류,남성의류상의,남성티셔츠
freq,NaN,NaN,NaN,NaN,NaN,NaN,652149,NaN,1177832,NaN,NaN,NaN,1248822,1811942,1834378,243234,141301,112344
mean,3.331127e+06,5.771997e+06,4.643213e+05,5.171409e+04,1.177213e+00,2.018065e+07,NaN,2.018065e+07,NaN,1.808029e+01,2.457046e+01,4.636881e+01,NaN,NaN,NaN,NaN,NaN,NaN
std,1.920446e+06,3.143726e+06,2.453568e+05,1.170307e+05,2.671846e+00,1.675485e+02,NaN,1.675485e+02,NaN,5.691230e+00,5.838839e+00,3.485762e+01,NaN,NaN,NaN,NaN,NaN,NaN
min,2.500000e+01,8.400000e+01,3.000000e+00,1.000000e+01,1.000000e+00,2.018040e+07,NaN,2.018040e+07,NaN,0.000000e+00,7.000000e+00,3.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.662513e+06,2.754364e+06,2.637300e+05,1.290000e+04,1.000000e+00,2.018051e+07,NaN,2.018051e+07,NaN,1.500000e+01,2.200000e+01,2.700000e+01,NaN,NaN,NaN,NaN,NaN,NaN
50%,3.334233e+06,5.678837e+06,4.774710e+05,2.800000e+04,1.000000e+00,2.018063e+07,NaN,2.018063e+07,NaN,1.900000e+01,2.500000e+01,3.900000e+01,NaN,NaN,NaN,NaN,NaN,NaN
75%,5.006782e+06,8.398250e+06,6.861755e+05,5.000000e+04,1.000000e+00,2.018082e+07,NaN,2.018082e+07,NaN,2.200000e+01,2.800000e+01,5.700000e+01,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
project_main.groupby("px_px").count()

,CLNT_ID,SESS_ID,PD_C,PD_BUY_AM,PD_BUY_CT,SESS_DT,ZON_NM,DATE,WEATHER,T_MAX,T_MIN,px_max,황사,비,CLAC1_NM,CLAC2_NM,CLAC3_NM
px_px,,,,,,,,,,,,,,,,,
NA,12734,12734,12734,12734,12734,12734,12734,12734,12734,12734,12734,0,12734,12734,12734,12734,12734
나쁨,68180,68180,68180,68180,68180,68180,68180,68180,68180,68180,68180,68180,68180,68180,68180,68180,68180
매우나쁨,41727,41727,41727,41727,41727,41727,41727,41727,41727,41727,41727,41727,41727,41727,41727,41727,41727
민감,431433,431433,431433,431433,431433,431433,431433,431433,431433,431433,431433,431433,431433,431433,431433,431433,431433
보통,1847371,1847371,1847371,1847371,1847371,1847371,1847371,1847371,1847371,1847371,1847371,1847371,1847371,1847371,1847371,1847371,1847371
좋음,2623461,2623461,2623461,2623461,2623461,2623461,2623461,2623461,2623461,2623461,2623461,2623461,2623461,2623461,2623461,2623461,2623461


# 특정 날씨(미세먼지 민감, 나쁨, 매우 나쁨일때 최다 구매 상품군 파악하기)

In [37]:
px_Y = project_main[project_main.px_max >=100]

px_N = project_main[project_main.px_max <100]

## 특정 날씨 / 전체 비율

In [38]:
print("미세 먼지 민감 기준이상일때 비율 :",px_Y.shape[0]/project_main.shape[0]*100 ,"%")

미세 먼지 민감 기준이상일때 비율 : 11.116546259770828 %


# 대분류 별 TOP 15

In [39]:
CLAC1 = px_Y.groupby("CLAC1_NM").count().sort_values("CLNT_ID", ascending=[False])[['CLNT_ID']][0:10]

In [40]:
CLAC1['per']= CLAC1.CLNT_ID / px_Y.shape[0] *100 

In [41]:
CLAC1

,CLNT_ID,per
CLAC1_NM,,
여성의류,76304,13.659962
스포츠패션,66373,11.882112
남성의류,64568,11.558980
화장품/뷰티케어,59880,10.719733
패션잡화,51494,9.218469
속옷/양말/홈웨어,47147,8.440268
유아동의류,42601,7.626442
출산/육아용품,15609,2.794327
시즌스포츠,14620,2.617276


# 중분류 별 TOP 10

In [42]:
CLAC2 = px_Y.groupby("CLAC2_NM").count().sort_values("CLNT_ID", ascending=[False])[['CLNT_ID']][0:10]

In [43]:
CLAC2['per']= CLAC2.CLNT_ID / px_Y.shape[0] *100 

In [44]:
CLAC2

,CLNT_ID,per
CLAC2_NM,,
남성의류상의,45910,8.218820
여성의류상의,36588,6.549993
메이크업,26948,4.824238
스킨케어,22978,4.113527
여성속옷,17288,3.094902
여성의류전신,16131,2.887776
여성의류하의,15836,2.834965
남성의류하의,14667,2.625690
수영/물놀이,14620,2.617276


# 소분류 별 TOP 10

In [45]:
CLAC3 = px_Y.groupby("CLAC3_NM").count().sort_values("CLNT_ID", ascending=[False])[['CLNT_ID']][0:10]

In [46]:
CLAC3['per']= CLAC3.CLNT_ID / px_Y.shape[0] *100 

In [47]:
CLAC3

,CLNT_ID,per
CLAC3_NM,,
남성티셔츠,37464,6.706815
여성티셔츠/탑,16592,2.970304
여성원피스,15868,2.840693
여성남방셔츠,13203,2.363604
남성캐주얼바지,11909,2.131952
여성바지,9344,1.672765
스킨케어세트,7000,1.253142
영유아티셔츠/탑,6951,1.244370
남성스포츠티셔츠,6850,1.226289
